# RBC Catalyst Matrix Factorization Workshop 2018

## Part 1 - Data Processing 

## The MovieLens Dataset
One of the most common datasets that is used for building and learning about Recommender Systems is the [MovieLens DataSet](https://grouplens.org/datasets/movielens/). This version of the dataset that we'll be working with ([1M](https://grouplens.org/datasets/movielens/1m/)) contains 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users who joined MovieLens in 2000.

Users were selected at random for inclusion. All users selected had rated at least 20 movies. Each user is represented by an id.

The original data are contained in three files, [movies.dat](https://github.com/khanhnamle1994/movielens/blob/master/dat/movies.dat), [ratings.dat](https://github.com/khanhnamle1994/movielens/blob/master/dat/ratings.dat) and [users.dat](https://github.com/khanhnamle1994/movielens/blob/master/dat/users.dat). 

To make it easier to work with the data we will convert them into csv's. 

In [ ]:
# Import packages
import os
import pandas as pd

In [ ]:
# Define file directories
MOVIELENS_DIR = 'dat'
USER_DATA_FILE = 'users.dat'
MOVIE_DATA_FILE = 'movies.dat'
RATING_DATA_FILE = 'ratings.dat'
# Specify User's Age and Occupation Column
AGES = { 1: "Under 18", 18: "18-24", 25: "25-34", 35: "35-44", 45: "45-49", 50: "50-55", 56: "56+" }
OCCUPATIONS = { 0: "other or not specified", 1: "academic/educator", 2: "artist", 3: "clerical/admin",
                4: "college/grad student", 5: "customer service", 6: "doctor/health care",
                7: "executive/managerial", 8: "farmer", 9: "homemaker", 10: "K-12 student", 11: "lawyer",
                12: "programmer", 13: "retired", 14: "sales/marketing", 15: "scientist", 16: "self-employed",
                17: "technician/engineer", 18: "tradesman/craftsman", 19: "unemployed", 20: "writer" }

In [ ]:
# Define csv files to be saved into
USERS_CSV_FILE = 'users.csv'
MOVIES_CSV_FILE = 'movies.csv'
RATINGS_CSV_FILE = 'ratings.csv'
# Read the Ratings File
ratings = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['user_id', 'movie_id', 'rating', 'timestamp'])

In [ ]:
# Set max_userid to the maximum user_id in the ratings
max_userid = ratings['user_id'].drop_duplicates().max()
# Set max_movieid to the maximum movie_id in the ratings
max_movieid = ratings['movie_id'].drop_duplicates().max()

# Process ratings dataframe for Keras Deep Learning model
# Add user_emb_id column whose values == user_id - 1
ratings['user_emb_id'] = ratings['user_id'] - 1
# Add movie_emb_id column whose values == movie_id - 1
ratings['movie_emb_id'] = ratings['movie_id'] - 1

print(str(len(ratings)) + ' ratings loaded')

In [ ]:
# Save into ratings.csv
ratings.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['user_id', 'movie_id', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])

print('Saved to ' + RATINGS_CSV_FILE)

In [ ]:
# Read the Users File
users = pd.read_csv(os.path.join(MOVIELENS_DIR, USER_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['user_id', 'gender', 'age', 'occupation', 'zipcode'])
users['age_desc'] = users['age'].apply(lambda x: AGES[x])
users['occ_desc'] = users['occupation'].apply(lambda x: OCCUPATIONS[x])

print(str(len(users)) + ' descriptions of ' + str(max_userid) + ' users loaded.')

In [ ]:
# Save into users.csv
users.to_csv(USERS_CSV_FILE, 
             sep='\t', 
             header=True, 
             encoding='latin-1',
             columns=['user_id', 'gender', 'age', 'occupation', 'zipcode', 'age_desc', 'occ_desc'])

print('Saved to ' + USERS_CSV_FILE)

In [ ]:

# Read the Movies File
movies = pd.read_csv(os.path.join(MOVIELENS_DIR, MOVIE_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movie_id', 'title', 'genres'])
print(str(len(movies)) + ' descriptions of ' + str(max_movieid) + ' movies loaded.')

In [ ]:
# Save into movies.csv
movies.to_csv(MOVIES_CSV_FILE, 
              sep='\t', 
              header=True, 
              columns=['movie_id', 'title', 'genres'])

print('Saved to ' + MOVIES_CSV_FILE)

## Part 2 - Data Preparation

We will load the data into pandas dataframes to make it easy for us to further explore the data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])

# Reading users file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

Now lets take a peak into the content of each file to understand them better.

### Ratings Dataset

In [ ]:
# Check the top 5 rows
ratings.head()

In [ ]:
# Check the file info
ratings.info()

### Users Dataset

In [ ]:
# Check the top 5 rows
users.head()

In [ ]:
# Check the file info
users.info()

### Movies Dataset

In [ ]:
# Check the top 5 rows
movies.head()

In [ ]:
# Check the file info
movies.info()

This dataset contains attributes of the 3883 movies. There are 3 columns including the movie ID, their titles, and their genres. Genres are pipe-separated and are selected from 18 genres (Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western).

## Part 3 - Data Exploration

### Titles

This is mostly textual data. So we can do some of our exploration visually to see if there are there certain words that feature more often in Movie Titles. We can use a word-cloud visualization.

In [ ]:
# Import new libraries
%matplotlib inline
import wordcloud
from wordcloud import WordCloud, STOPWORDS

# Create a wordcloud of the movie titles
movies['title'] = movies['title'].fillna("").astype('str')
title_corpus = ' '.join(movies['title'])
title_wordcloud = WordCloud(stopwords=STOPWORDS, background_color='black', height=2000, width=4000).generate(title_corpus)

# Plot the wordcloud
plt.figure(figsize=(16,8))
plt.imshow(title_wordcloud)
plt.axis('off')
plt.show()

We can see some important information about this data. There are a lot of movie franchises in this dataset, as evidenced by words like *II* and *III*. In addition to that, *Day*, *Love*, *Life*, *Time*, *Night*, *Man*, *Dead*, *American* are among the most commonly occuring words.

### Ratings
Next let's examine the **rating** data further. Let's take a look at its summary statistics and distribution.

In [ ]:
# Get summary statistics of rating
ratings['rating'].describe()

In [ ]:
# Import seaborn library
import seaborn as sns
sns.set_style('whitegrid')
sns.set(font_scale=1.5)
%matplotlib inline

# Display distribution of rating
sns.distplot(ratings['rating'].fillna(ratings['rating'].median()))

It appears that users are quite generous in their ratings. The mean rating is 3.58 on a scale of 5. Half the movies have a rating of 4 and 5. Some questions we can ask ourselves about this:

What does a 5 rating mean given the distribution? 

I personally think that a 5-level rating skill wasn’t a good indicator as people could have different rating styles (i.e. person A could always use 4 for an average movie, whereas person B only gives 4 out for their favorites). Each user rated at least 20 movies, so I doubt the distribution could be caused just by chance variance in the quality of movies.

Let's also take a look at a subset of 20 movies with the highest rating.

In [ ]:
# Join all 3 files into one dataframe
dataset = pd.merge(pd.merge(movies, ratings),users)
# Display 20 movies with highest ratings
dataset[['title','genres','rating']].sort_values('rating', ascending=False).head(20)

### Genres
Genre should be great categorical variable to explore. Intuitively we can think that it should serve as a good descriptor of the contents of a movie serving as a simple similarity measure. Now similarity really matters when building a recommendation engine. A basic assumption is that films in the same genre should have similar contents. 

In [ ]:
# Make a census of the genre keywords
genre_labels = set()
for s in movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

# Function that counts the number of times each of the genre keywords appear
def count_word(dataset, ref_col, census):
    keyword_count = dict()
    for s in census: 
        keyword_count[s] = 0
    for census_keywords in dataset[ref_col].str.split('|'):        
        if type(census_keywords) == float and pd.isnull(census_keywords): 
            continue        
        for s in [s for s in census_keywords if s in census]: 
            if pd.notnull(s): 
                keyword_count[s] += 1
    #______________________________________________________________________
    # convert the dictionary in a list to sort the keywords by frequency
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count

# Calling this function gives access to a list of genre keywords which are sorted by decreasing frequency
keyword_occurences, dum = count_word(movies, 'genres', genre_labels)
keyword_occurences[:5]

The top 5 genres are: 

* Drama
* Comedy
* Action
* Thriller
* Romance

## Part 2 - Recommendation Engines

### Content Based Filtering Implementation
We will build a Content-Based Recommendation Engine that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

In [ ]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
# Convert genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')

We'll use **TfidfVectorizer** function from **scikit-learn**, which transforms text to feature vectors that can be used as input to the estimator.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

We will be using [Cosine Similarity](https://masongallo.github.io/machine/learning,/python/2016/07/29/cosine-similarity.html)** to calculate a numeric quantity that denotes the similarity between two movies. Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's **linear_kernel** instead of cosine_similarities since it is much faster.

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

This produces a pairwise cosine similarity matrix for all the movies in the dataset. The next step is to write a function that returns the 20 most similar movies based on the cosine similarity score.

In [ ]:
# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Let's try and get the top recommendations for a few movies and see how good the recommendations are.

In [ ]:
genre_recommendations('Good Will Hunting (1997)').head(20)

In [ ]:
genre_recommendations('Toy Story (1995)').head(20)

In [ ]:
genre_recommendations('Saving Private Ryan (1998)').head(20)

As you can see, I have quite a decent list of recommendation for **Good Will Hunting** (Drama), **Toy Story** (Animation, Children's, Comedy), and **Saving Private Ryan** (Action, Thriller, War).

### Collaborative Filtering Recommendation Model

### Implementation
Let's use the **ratings.csv** file first as it contains User ID, Movie IDs and Ratings. 

In [ ]:
# Fill NaN values in user_id and movie_id column with 0
ratings['user_id'] = ratings['user_id'].fillna(0)
ratings['movie_id'] = ratings['movie_id'].fillna(0)

# Replace NaN values in rating column with average of all values
ratings['rating'] = ratings['rating'].fillna(ratings['rating'].mean())

We'll take a subset of the data to do a quick analysis here. We can take a random sample of 20,000 ratings (2%) from the 1M ratings.

In [ ]:
# Randomly sample 1% of the ratings dataset
small_data = ratings.sample(frac=0.02)
# Check the sample info
print(small_data.info())

Now we will split our data into training and testing sets. We can then do cross-validation after. **Cross_validation.train_test_split** shuffles and splits the data into two datasets according to the percentage of test examples, which in this case is 0.2.

In [ ]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(small_data, test_size=0.2)

Now we need to create a user-item matricies (one for training and one for testing). 

In [ ]:
# Create two user-item matrices, one for training and another for testing
train_data_matrix = train_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])
test_data_matrix = test_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])

# Check their shape
print(train_data_matrix.shape)
print(test_data_matrix.shape)

We can then use the **pairwise_distances** function from sklearn to calculate the [Pearson Correlation Coefficient](https://stackoverflow.com/questions/1838806/euclidean-distance-vs-pearson-correlation-vs-cosine-similarity). This method provides a safe way to take a distance matrix as input, while preserving compatibility with many other algorithms that take a vector array.

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])

In [ ]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation[:4, :4])

With the similarity matricies in hand, we can now predict the ratings that were not included with the data. This is where we have all the fun.

For the user-user CF case, we will look at the similarity between 2 users (A and B, for example) as weights that are multiplied by the ratings of a similar user B (corrected for the average rating of that user). We also need to normalize it so that the ratings stay between 1 and 5 and, as a final step, sum the average ratings for the user that we are trying to predict. 

The idea here is that some users may tend always to give high or low ratings to all movies. The relative difference in the ratings that these users give is more important than the absolute values. 

In [ ]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

### Evaluation

There are many evaluation metrics but one of the most popular metric used to evaluate accuracy of predicted ratings is **Root Mean Squared Error (RMSE)**.

The root-mean-square deviation (RMSD) or root-mean-square error (RMSE) (or sometimes root-mean-squared error) is a frequently used measure of the differences between values (sample or population values) predicted by a model or an estimator and the values observed.

It is calculated as follows:

$$\mathit{RMSE} =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}$$

We can use scikit-learn's **mean squared error** function as our validation metric. Comparing user- and item-based collaborative filtering, it looks like user-based collaborative filtering gives a better result.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [ ]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

# RMSE on the test data
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

In [ ]:
# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix)))

You'll notice that our RMSE is quite big. This most likely indicates that our training model is overfit to the data. How can we remedy this?

We've just implemented a Memory-based Collaborative Filter. However, there are some drawback to this approach:

* It doesn't address the well-known cold-start problem, that is when new user or new item enters the system. 
* It can't deal with sparse data, meaning it's hard to find users that have rated the same items.
* It suffers when new users or items that don't have any ratings enter the system.
* It tends to recommend popular items.

## Part 3- Exploring Basic Matrix Factorization

### Matrix Decomposition

Let's go through a very basic Matrix Decomposition to understand what's happening. We will use a type of decomposition called LU Decomposition. https://en.wikipedia.org/wiki/LU_decomposition

This will allow us to observe a matrix decomposition. 

In [ ]:
# LU decomposition
from numpy import array
from scipy.linalg import lu

In [ ]:
# define a square matrix
A = array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(A)

In [ ]:
# LU decomposition
P, L, U = lu(A)
print(P)
print(L)
print(U)

In [ ]:
# reconstruct
B = P.dot(L).dot(U)
print(B)

As you can see, it's quite easy to break down this matrix into smaller components 

### SVD

Now we will attempt to do Singular Value Decomposition. https://en.wikipedia.org/wiki/Singular-value_decomposition

In [ ]:
from numpy import array
from scipy.linalg import svd
from numpy import diag
from numpy import dot
from numpy import zeros

In [ ]:
# Singular-value decomposition
# define a matrix
A = array([[1, 2], [3, 4], [5, 6]])
print(A)
# SVD
U, s, VT = svd(A)
print(U)
print(s)
print(VT)

In [ ]:
# create m x n Sigma matrix
Sigma = zeros((A.shape[0], A.shape[1]))
# populate Sigma with n x n diagonal matrix
Sigma[:A.shape[1], :A.shape[1]] = diag(s)
# reconstruct matrix
B = U.dot(Sigma.dot(VT))
print(B)

Keep in mind this is being done on a very small matrix. Imagine this process being done on a very large matrix each time and you can quickly imagine the scalability problem. 

## SVD on MovieLens Data 
Let's load the 3 data files just like last time.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd

# Reading ratings file
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating', 'timestamp'])

# Reading users file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

Let's take a look at the movies and ratings dataframes.

In [ ]:
movies.head()

In [ ]:
ratings.head()

Also let's count the number of unique users and movies.

In [ ]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print( 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Let's checkout how sparse our matrix is. 

In [ ]:
sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

Now we will format the ratings matrix to be one row per user and one column per movie. To do so, we can pivot *ratings* to get and call the new variable *Ratings* (with a capital *R).

In [ ]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

Last but not least, we should de-normalize the data (normalize by each users mean) and convert it from a dataframe to a numpy array.

In [ ]:
R = Ratings.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

With our ratings matrix properly formatted and normalized, we should be ready to try out SVD.

### Setting Up SVD
Scipy and Numpy both have functions to do the singular value decomposition.

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

As we will leverage matrix multiplication to get predictions, we want to convert the $\Sigma$ (now are values) to the diagonal matrix form.

In [ ]:
sigma = np.diag(sigma)

### Making Predictions from the Decomposed Matrices
We now have everything we need to make movie ratings predictions for every user. We can do it all at once by following the math and matrix multiply $U$, $\Sigma$, and $V^{T}$ back to get the rank $k=50$ approximation of $A$.

But first, we will need to add the user means back to get the actual star ratings prediction.

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

With the predictions matrix for every user, we can build a function to recommend movies for any user. We can then return the list of movies the user has already rated, for the sake of comparison.

In [ ]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

Now we get to the fun stuff...let's build a function to return the movies with the highest predicted rating that the specified user hasn't already rated. Notice we haven't used any explicit movie content features (such as genre or title). We will merge in that information to get a more complete picture of the recommendations.

In [ ]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

Let's try to recommend 20 movies for user with ID 1310.

In [ ]:
already_rated, predictions = recommend_movies(preds, 1310, movies, ratings, 20)

In [ ]:
# Top 20 movies that User 1310 has rated 
already_rated.head(20)

In [ ]:
# Top 20 movies that User 1310 hopefully will enjoy
predictions

These look like pretty good recommendations. It's good to see that, although we didn't actually use the genre of the movie as a feature, the truncated matrix factorization features "picked up" on the underlying tastes and preferences of the user. 

### Model Evaluation
Of course we can't say for sure that we've developed a good model without doing a basic evaluation

Here we will use the *[Surprise](https://pypi.python.org/pypi/scikit-surprise)* library that provided various ready-to-use powerful prediction algorithms including (SVD) to evaluate its RMSE (Root Mean Squared Error) on the MovieLens dataset. It is a Python scikit building and analyzing recommender systems.

In [ ]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD, evaluate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Split the dataset for 5-fold evaluation
data.split(n_folds=5)

In [ ]:
# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm.
evaluate(svd, data, measures=['RMSE'])

We get a mean *Root Mean Square Error* of 0.8736 which is pretty good. Let's now train on the dataset and arrive at predictions.

In [ ]:
trainset = data.build_full_trainset()
svd.train(trainset)

Now let's use SVD to predict the rating that User with ID 1310 will give to a random movie (let's say with Movie ID 1994).

In [ ]:
ratings[ratings['user_id'] == 1310]

In [ ]:
svd.predict(1310, 1994)

For movie with ID 1994, we get an estimated prediction of 3.349. The recommender system works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

## Part 4 - Using Surprise for Actual Prediction

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
file_path = 'user.csv'

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating timestamp', sep='\t')

data = Dataset.load_from_file(file_path, reader=reader)

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

## Part 4 - Using Lenskit

In [ ]:
import pandas as pd
from lenskit import batch, topn
from lenskit import crossfold as xf
from lenskit.algorithms import item_knn

In [ ]:
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user', 
                                                         'item', 
                                                         'rating', 
                                                         'timestamp'])
algo = item_knn.ItemItem(30)

In [ ]:
def eval(train, test):
    model = algo.train(train)
    users = test.user.unique()
    recs = batch.recommend(algo, model, users, 100, topn.UnratedCandidates(train))
    # combine with test ratings for relevance data
    res = pd.merge(recs, test, how='left', on=('user', 'item'))
    # fill in missing 0s
    res.loc[res.rating.isna(), 'rating'] = 0
    return res

In [ ]:
# compute evaluation
splits = xf.partition_users(ratings, 5, xf.SampleFrac(0.2))
recs = pd.concat((eval(train, test) for (train, test) in splits))

In [ ]:
# compile results
ndcg = recs.groupby('user').rating.apply(topn.ndcg)

In [ ]:
ndcg

Can you figure out how to calculate RMSE to evaluate performance?

# References

This workshop was developed using several excellent resources. The following are the major contributors:

1) Primarly James Le's "The 4 Recommendation Engines That Can Predict Your Movie Tastes" :

https://towardsdatascience.com/the-4-recommendation-engines-that-can-predict-your-movie-tastes-109dc4e10c52
https://github.com/khanhnamle1994/movielens

2) The following blog posts by Introduction to Machine Learning:

https://machinelearningmastery.com/introduction-to-matrix-decompositions-for-machine-learning/
https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/

3) Albert Au Yeung's Matrix Factorization Tutorial:

http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/

4) Chris Clark's "A Simple Content-Based Recommendation Engine in Python" blog post:

http://blog.untrod.com/2016/06/simple-similar-products-recommendation-engine-in-python.html

5) Daryl Lim's "Matrix Factorization and Collaborative Filtering Lecture:

http://acsweb.ucsd.edu/~dklim/mf_presentation.pdf

6) Simon Funk's FunkSVD Blog Post:

http://sifter.org/~simon/journal/20061211.html

7) Prince Grover's "Various Implementations of Collaborative Filtering":

https://towardsdatascience.com/various-implementations-of-collaborative-filtering-100385c6dfe0

8) Nicolas Hug's "Understanding matrix factorization for recommendation":

http://nicolas-hug.com/blog/matrix_facto_1
http://nicolas-hug.com/blog/matrix_facto_2
http://nicolas-hug.com/blog/matrix_facto_3
http://nicolas-hug.com/blog/matrix_facto_4

9) G. Linden ; B. Smith ; J. York "Amazon.com recommendations: item-to-item collaborative filtering":

https://ieeexplore.ieee.org/document/1167344/

10) Xavier Amatriain and Justin Basilico "Netflix Recommendations - Beyond the 5 Stars" presentation and blog posts:

https://www.slideshare.net/xamat/netflix-recommendations-beyond-the-5-stars
https://medium.com/netflix-techblog/netflix-recommendations-beyond-the-5-stars-part-1-55838468f429
https://medium.com/netflix-techblog/netflix-recommendations-beyond-the-5-stars-part-2-d9b96aa399f5

11) CARLOS A. GOMEZ-URIBE and NEIL HUNT, Netflix, Inc. "The Netflix Recommender System: Algorithms, Business Value,
and Innovation":

http://delivery.acm.org/10.1145/2850000/2843948/a13-gomez-uribe.pdf?ip=142.245.59.18&id=2843948&acc=OA&key=4D4702B0C3E38B35%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35%2EE5B8A747884E71D5&__acm__=1537803530_7caa794481476feae49c4ae12b8e0c7c
